# 神経細胞間の接続
第1章ではニューロンのモデルについて、第2章ではシナプスのモデルについて学んできました。第3章ではそれらのモデルを組み合わせたネットワークを構築してみます。また、最後の節ではSNNを学習させる意義とその方針について説明します。

### ニューロン間の接続
ネットワークを構成するにはあるニューロンがどのニューロンに投射しているか、どのように活動が伝搬するかを記述する必要があります。この節ではニューロン同士の間の接続関係の記述の仕方について説明します。

### 全結合(Full connection)
$i$層目のニューロンが$i+1$層目のニューロンに全て繋がっていることを**全結合** (fully connected)と言います。ただし、全てが完全に繋がっているということではなく、結合重みが0の場合は繋がっていないことを表します。なお、この結合様式は既に第2章で出てきています。全結合はANNでは入力に重み行列を乗算し、バイアスを加算するようなアフィン変換で表されますが、SNNでは入力に重み行列を乗算するだけの線形変換を用いることが主です。

単に重み行列を用意するだけでも(この本の内容に限るなら)問題はありませんが、重みを学習させる場合には`class`を用意しておくと取り扱いがしやすくなります。コードは次のようになります

In [ ]:
class FullConnection:
    def __init__(self, N_in, N_out, initW=None):
        if initW is not None:
            self.W = initW
        else:
            self.W = 0.1*np.random.rand(N_out, N_in)
    
    def backward(self, x):
        return np.dot(self.W.T, x) #self.W.T @ x
    
    def __call__(self, x):
        return np.dot(self.W, x) #self.W @ x

### 2次元の畳み込み(Convolution2D connection)
SNNではANNの1つの結合形式である\textbf{畳み込み層}(convolutional layer)を含むことがあります。全結合が通常のANNと同様であったように畳み込み層も全く同じ操作です。そのため、今回実装はしないのですが、行列計算ライブラリとしてNumPyではなく、TensorflowやPytorch, Chainer等を使う場合には畳み込み層の関数が実装されているのでそれを使うとよいでしょう。\par
念のため、2D畳み込み層の出力テンソル($H\times W \times C$のテンソル、$H, W$はそれぞれ画像の高さと幅、$C$はチャネル数)の解釈について説明しておきます。まず、1つのチャネルは同種(同系統の受容野を持つ)の$H\times W$個のニューロンの活動です。本来は「同種」ですが、空間的な不変性により「同一」と見なし、重み共有(weight sharing, weight tying)をしてスライディングウィンドウ(sliding window)の操作をすることで、ニューロンを視野全体に複製(要は1つのニューロンをコピペ)しています。実際の視覚野では近傍のニューロンの活動を受けることによる畳み込みはしていますが、重み共有\footnote{ただし、類似の遺伝子発現による初期値共有はしているかもしれないですが。}とスライディングウィンドウはしていない、ということです。

### 遅延結合(Delay connection)
実際のニューロンにおいて、シナプス前細胞での発火が瞬間的にシナプス後細胞に伝わるということはありません。これは**軸索遅延** (axonal delays)や**シナプス遅延** (synaptic delay)があるためです。ここでは発火情報の伝搬における遅延の実装について説明します。ただし、全てのニューロンの遅延が等しいとした場合のみです\footnote{遅延時間をバラバラにすると行列での取り扱いが難しくなり、for loopを用いる他にないと思うので省略します。実装したい場合はC++やJuliaなどfor loopが速い言語を用いてください。}。\par
実装は単純で、まず、行数はニューロンの数、列数は遅延時間のステップ数と同じ長さとした行列を用意します。ステップごとに最後の行にあたるベクトルを出力し、配列をずらした後\footnote{\texttt{np.roll}を用いるよりもこちらの方が速いです。}、初めの行を新しい入力で更新します。

In [ ]:
class DelayConnection:
    def __init__(self, N, delay, dt=1e-4):
        nt_delay = round(delay/dt) # 遅延のステップ数
        self.state = np.zeros((N, nt_delay))
        
    def __call__(self, x):
        out = self.state[:, -1] # 出力
        self.state[:, 1:] = self.state[:, :-1] # 配列をずらす
        self.state[:, 0] = x # 入力
        return out

このコードも\texttt{./Models/Connections.py}に記して保存しておきましょう。\par
次に、遅延が正しく表現されているか確認してみましょう\footnote{コードは\texttt{./TrainingSNN/example\_using\_delay\_connection.py}です。また、この部分はBrian2のtutorialを参考にしました。}。まず、\texttt{Models}ディレクトリをパッケージとして認識させるために\texttt{\_\_init\_\_.py}という名称のファイルを作成し(何も書かれてなくてよいです)、\texttt{Models}ディレクトリ内に保存します。次に\texttt{Models}ディレクトリの親ディレクトリ内にこれから書くファイルを置きます。こうすることで\texttt{Models}ディレクトリ内のファイルから作成した\texttt{class}をimportすることができます。これは以降のコードでほぼ共通です。\par
さて、コードは次のようになります。初めにモデルのimportと定数の定義、モデルのインスタンスの作成、記録用配列の定義を行っています。

In [ ]:
from Models.Neurons import CurrentBasedLIF
from Models.Connections import DelayConnection

dt = 1e-4; T = 5e-2; nt = round(T/dt)

#モデルの定義
neuron1 = CurrentBasedLIF(N=1, dt=dt, tc_m=1e-2, tref=0, 
                          vrest=0, vreset=0, vthr=1, vpeak=1)
neuron2 = CurrentBasedLIF(N=1, dt=dt, tc_m=1e-1, tref=0,
                          vrest=0, vreset=0, vthr=1, vpeak=1)
delay_connect = DelayConnection(N=1, delay=2e-3, dt=dt)

I = 2 # 入力電流
v_arr1 = np.zeros(nt); v_arr2 = np.zeros(nt) #記録用配列

for t in tqdm(range(nt)):
    # 更新
    s1 = neuron1(I)
    d1 = delay_connect(s1)
    s2 = neuron2(0.02/dt*d1)

    # 保存
    v_arr1[t] = neuron1.v_
    v_arr2[t] = neuron2.v_

time = np.arange(nt)*dt*1e3
plt.figure(figsize=(5, 4))
plt.plot(time, v_arr1, label="Neuron1", linestyle="dashed")
plt.plot(time, v_arr2, label="Neuron2")
plt.xlabel("Time (ms)"); plt.ylabel("v") 
plt.legend(loc="upper left")
plt.show()

ニューロン1からニューロン2へと2 msの遅延で発火が伝わる場合。